1.泡点温度、泡点压力的计算测试

2.闪蒸测试
测试两相TP闪蒸方法VLE_TPflash()

注：本案例含有聚合物, 因此不涉及露点温度和露点压力

泡点计算的Aspen对照文件: 

闪蒸计算的Aspen对照文件: 
..\polymer_model\thermodynamics\test\Aspen\pp_flash
..\polymer_model\thermodynamics\test\Aspen\pp_flash1
..\polymer_model\thermodynamics\test\Aspen\hdpe_flash

In [11]:
from polymer_model.thermodynamics.flash_algorithm import *
import numpy as np

In [12]:
# 泡点温度、压力计算
# case 1: "C3H6", "C3H8", "H2", "N2", "PP", "TICL4", "TEA"
T = 342.15
P = 2800000
PP_DPN = 535.4335 
PP_MWN = 42.0806*PP_DPN
z = np.array([0.2098286, 0.00408862, 0.0005843, 0.000137371, 0.7851512/PP_DPN,
                3.22413E-05, 0.000177694])
z = z/np.sum(z)
args = {"Tc": np.array([364.85, 369.83, 33.19, 126.2, 2000, 638, 678.15]), 
        "Pc": np.array([4600000, 4248000, 1313000, 3400000, 5000000, 4660950, 8930000]), 
        "w": np.array([0.137588, 0.152291, -0.215993, 0.0377215, 0, 0.283732, 0.841783]),
        "m": np.array([1.9597, 2.002, 0.9846, 1.2053, 0.02528*PP_MWN, 20, 20]),
        "s": np.array([3.5356, 3.6184, 2.8263, 3.313, 4.1473, 3.54, 3.54]),
        "e": np.array([207.19, 208.11, 20.893, 90.96, 298.6392, 210, 210])}
Pb = bubble_pressure(T, z, args)
print("泡点压力: ", Pb)
Tb = bubble_temperature(P, z, args)
print("泡点温度：", Tb)

泡点压力:  (2728939.7220213343, array([8.98029950e-01, 1.72185017e-02, 7.61300088e-02, 8.62153739e-03,
       0.00000000e+00, 1.12343172e-14, 6.19165718e-14]), 2, 7)
泡点温度： (343.7129368650262, array([9.01264321e-01, 1.73045087e-02, 7.30713590e-02, 8.35978215e-03,
       0.00000000e+00, 1.66071657e-14, 9.15283718e-14]), 3, 5)


In [13]:
# 案例1中聚合物PP含量少, 迭代12次后就收敛了
# Flash test 1: "C3H6", "C3H8", "H2", "N2", "PP", "TICL4", "TEA"
T = 333.15
P = 3000000
PP_MWN = 76386.77
PP_DPN = 1815.247
# z = np.array([0.7682945, 0.143522, 0.0784859, 0.00720122, 0.00249226/PP_DPN, 6.25573E-07, 3.46464E-06])
z = np.array([7.682945e-01, 1.435220e-01, 7.848590e-02, 7.201220e-03, 2.492260e-03/PP_DPN,
                6.255730e-07, 3.464640e-06])

z = z/np.sum(z)
print("z", z)
args = {"Tc": np.array([364.85, 369.83, 33.19, 126.2, 2000, 638, 678.15]), 
        "Pc": np.array([4600000, 4248000, 1313000, 3400000, 5000000, 4660950, 8930000]), 
        "w": np.array([0.137588, 0.152291, -0.215993, 0.0377215, 0, 0.283732, 0.841783]),
        "m": np.array([1.9597, 2.002, 0.9846, 1.2053, 0.02528*PP_MWN, 20, 20]),
        "s": np.array([3.5356, 3.6184, 2.8263, 3.313, 4.1473, 3.54, 3.54]),
        "e": np.array([207.19, 208.11, 20.893, 90.96, 298.6392, 210, 210])}

beta, x, y, iter_total = VLE_TPflash(T, P, z, args)
print("Vapor fraction: ", beta)
print("Liquid composition: ", x)
x[4] = x[4]*PP_DPN
beta = beta/(beta+(1-beta)*sum(x))
print("新气相分数：", beta)
x = x/sum(x)
print("新液相组成：", x)

print("Vapor composition: ", y)
print("iter_total: ", iter_total)

z [7.70213037e-01 1.43880394e-01 7.86818900e-02 7.21920243e-03
 1.37638763e-06 6.27135141e-07 3.47329168e-06]
Vapor fraction:  0.7092611438703768
Liquid composition:  [8.26054215e-01 1.65974718e-01 6.73919994e-03 1.21302931e-03
 4.73410163e-06 2.15703878e-06 1.19464281e-05]
新气相分数： 0.7074944548555515
新液相组成： [8.19019794e-01 1.64561329e-01 6.68181100e-03 1.20269953e-03
 8.52038366e-03 2.13867011e-06 1.18446960e-05]
Vapor composition:  [7.47322729e-01 1.34823532e-01 1.08172498e-01 9.68124130e-03
 0.00000000e+00 4.40134475e-16 2.43761720e-15]
iter_total:  17


In [14]:
# 案例2的聚合物含量高, 迭代到最大次数，最终结果和Aspen相近
# Flash test 2: "C3H6", "C3H8", "H2", "N2", "PP", "TICL4", "TEA"
T = 338.15
P = 500000
PP_DPN = 535.4335 
PP_MWN = 42.0806*PP_DPN
z = np.array([0.2098286, 0.00408862, 0.0005843, 0.000137371, 0.7851512/PP_DPN,
                3.22413E-05, 0.000177694])
# z = np.array([0.0251157, 0.000495333, 1.33715E-06, 7.41334E-07, 0.9741264/PP_DPN, 4.00013E-05, 0.000220463])

z = z/np.sum(z)
print("z", z)
args = {"Tc": np.array([364.85, 369.83, 33.19, 126.2, 2000, 638, 678.15]), 
        "Pc": np.array([4600000, 4248000, 1313000, 3400000, 5000000, 4660950, 8930000]), 
        "w": np.array([0.137588, 0.152291, -0.215993, 0.0377215, 0, 0.283732, 0.841783]),
        "m": np.array([1.9597, 2.002, 0.8285, 1.2053, 0.02528*PP_MWN, 20, 20]),
        "s": np.array([3.5356, 3.6184, 2.9729, 3.313, 4.1473, 3.5356, 3.5356]),
        "e": np.array([207.19, 208.11, 12.53, 90.96, 298.6392, 207.19, 207.19])}

beta, x, y, iter = VLE_TPflash(T, P, z, args)
print("Vapor fraction: ", beta)
print("Liquid composition: ", x)
x[4] = x[4]*PP_DPN
beta = beta/(beta+(1-beta)*sum(x))
print("新气相分数：", beta)
x = x/sum(x)
print("新液相组成：", x)

print("Vapor composition: ", y)
print("iter: ", iter)

z [9.70013156e-01 1.89012136e-02 2.70115078e-03 6.35050118e-04
 6.77892299e-03 1.49047771e-04 8.21458647e-04]
Vapor fraction:  0.8968137256565466
Liquid composition:  [9.06950027e-01 1.78870292e-02 3.77468499e-05 2.67711884e-05
 6.56934106e-02 1.44439706e-03 7.96061856e-03]
新气相分数： 0.19400038133036362
新液相组成： [2.51171750e-02 4.95365376e-04 1.04536546e-06 7.41404270e-07
 9.74125209e-01 4.00012929e-05 2.20462256e-04]
Vapor composition:  [9.77269437e-01 1.90179095e-02 3.00761208e-03 7.05040985e-04
 0.00000000e+00 4.32985781e-15 2.38634842e-14]
iter:  100


In [15]:
# 本案例迭代到最大次数，最终结果和Aspen相近
# Flash test 3: C2H4、H2、C6H14、HDPE、TEA、TICL4
T = 363.15 
P = 400000
DPN = 1093
MWN = 30662.76
z = np.array([0.0409126, 0.00126541, 0.9573389, 0.000442622/DPN, 3.93743E-05, 1.03188E-06])
z = z/np.sum(z)
args = {"Tc": np.array([282.34, 33.19, 507.6, 2000, 678.15, 638]), 
        "Pc": np.array([5041000, 1313000, 3025000, 5000000, 8930000, 4660950]), 
        "w": np.array([0.0862484, -0.215993, 0.301261, 0, 0.841783, 0.283732]),
        "m": np.array([1.5567, 0.98625, 3.0576, 0.05013*MWN, 25, 25]),
        "s": np.array([3.4358, 2.8242, 3.7983, 3.2336, 2.668, 2.668]),
        "e": np.array([179.53, 20.876, 236.77, 241.43, 198.8, 198.8])}

beta, x, y, iter_total = VLE_TPflash(T, P, z, args)
print("Vapor fraction: ", beta)
print("Liquid composition: ", x)
x[3] = x[3]*DPN
beta = beta/(beta+(1-beta)*sum(x))
print("新气相分数：", beta)
x = x/sum(x)
print("新液相组成：", x)

print("Vapor composition: ", y)
print("iter: ", iter)

Vapor fraction:  0.034759023365928905
Liquid composition:  [2.51002707e-02 1.56929072e-04 9.74700501e-01 4.19729210e-07
 4.08102453e-05 1.06951174e-06]
新气相分数： 0.03474365233298426
新液相组成： [2.50887714e-02 1.56857177e-04 9.74253957e-01 4.58553851e-04
 4.07915488e-05 1.06902176e-06]
Vapor composition:  [4.80534350e-01 3.20635119e-02 4.87402138e-01 0.00000000e+00
 1.19417117e-15 3.12955746e-17]
iter:  100
